In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-05-10 11:34:22,587 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [109]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_4.model')
model.init_sims(replace=True)

2017-05-10 12:42:10,125 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_4.model
2017-05-10 12:42:10,218 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_4.model.wv.* with mmap=None
2017-05-10 12:42:10,219 : INFO : setting ignored attribute syn0norm to None
2017-05-10 12:42:10,220 : INFO : setting ignored attribute cum_table to None
2017-05-10 12:42:10,221 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_4.model
2017-05-10 12:42:10,253 : INFO : precomputing L2-norms of word weight vectors


In [3]:
print(model.most_similar(positive=['rss', 'haha']))

[('d+', 0.9186103940010071), ('lindinha', 0.9171167016029358), ('Morri', 0.9156837463378906), ('*u*', 0.9115594625473022), ('Pqp', 0.9108421206474304), ('fofinho', 0.9079639911651611), ('ae,', 0.9064475893974304), ('hihi', 0.9053689241409302), (':p"', 0.9032702445983887), ('Haha"', 0.9031860828399658)]


In [4]:
num_features= 600

In [80]:
df = pd.read_csv('/home/bahbbc/personality-normalized-v2-word2vec.csv', encoding='utf-8')
df.shape

(1039, 186)

In [81]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [82]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [110]:
from sklearn.cluster import KMeans

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0


In [111]:
num_clusters = word_vectors.shape[0] / 10
num_clusters

1230

In [112]:
import time

start = time.time() # Start time

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

Time taken for K Means clustering:  287.718467951 seconds.


In [113]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [114]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids


In [90]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [91]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [92]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [93]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [94]:
train_tokenized[4]

[u'Que',
 u'maravilha',
 u'de',
 u'lugar',
 u'hein',
 u'Didi',
 u'Ardirene',
 u'Souza',
 u'Quero',
 u'voltar',
 u'Feliz',
 u'Ano',
 u'Novo',
 u'Feliz',
 u'Corinthians',
 u'salve',
 u'Corinthians',
 u'Vem',
 u'que',
 u'tem',
 u'Festa',
 u'do',
 u'Hawaii',
 u'calor',
 u'\xe1gua',
 u'de',
 u'c\xf4co',
 u'embora',
 u'Eduardo',
 u'Sousa',
 u'J\xe9ssica',
 u'Herrera',
 u'Esse',
 u'hor\xe1rio',
 u'de',
 u'ver\xe3o',
 u'acaba',
 u'comigo',
 u'huck',
 u'pregui\xe7a',
 u'huckbabao',
 u'dan\xe7a',
 u'Dos',
 u'Famosos',
 u'Faustao',
 u'Chama',
 u'Eu',
 u's\xedtio',
 u'anivers\xe1rio',
 u'family',
 u'anivers\xe1rio',
 u'da',
 u'bahiris',
 u'anivers\xe1rio',
 u'da',
 u'm\xe3e',
 u'Curtindo',
 u'Carioca',
 u'Club',
 u'do',
 u'lado',
 u'de',
 u'fora',
 u'fala',
 u'mansa',
 u'forrueiros',
 u'filosofia',
 u'reggae',
 u'lotado',
 u'povo',
 u'vai',
 u'viajar',
 u'Huck',
 u'estava',
 u'cansado',
 u'de',
 u'fazer',
 u'as',
 u'mesmas',
 u'coisas',
 u'Um',
 u'belo',
 u'dia',
 u'resolvi',
 u'mudar',
 u'Me',
 u

In [95]:
train_w2v_data['formatted_text'].head

<bound method Series.head of 118    Gabriel ?    As Melhores Mães do Mundo na mesm...
914    Papa Francisco nos pede para termos delicadeza...
250      O Startup Weekend que participei e ganhamos ...
390      quero , preciso    que ?   Cielusinsky kkk  ...
827    Que maravilha de lugar hein Didi Ardirene Souz...
377    eu cresci ouvindo essa música e só aos   anos ...
9      Johnny Taira  Thiago Luiz    It ' sim only for...
963    cooltest . me  Diego Alves embora lá . . .  po...
196    Vamos Leandro Rubens Sposito ? ?    To muito b...
887          Um feliz dia internacional da mulher par...
81       Retweeted Frases da Kéfera frases da Kefera ...
429    Rumo ao Banespa !  Do jogo de ontem . Criançad...
526    mudou muito não Marty !  kkk essa é a melhor d...
781      Cause baby , tonight , we ' re beautiful now...
239      Caráter é aquilo que você é , quando ninguém...
334    Vida .  Cauã Gimenez Caique Lucas Fernanda Pas...
211      Em época de pedidos de IMPTMA e intervenção ...
76

In [115]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train_w2v_data["formatted_text"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in train_tokenized:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test_w2v_data["formatted_text"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in test_tokenized:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [12]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-04-27 00:55:26,438 : WARNING : cannot compute similarity with no input [u'FOREIGN']
2017-04-27 00:55:26,763 : WARNING : cannot compute similarity with no input []
2017-04-27 00:55:27,051 : WARNING : cannot compute similarity with no input []
2017-04-27 00:55:27,057 : WARNING : cannot compute similarity with no input []
2017-04-27 00:55:27,616 : WARNING : cannot compute similarity with no input []
2017-04-27 00:55:27,840 : WARNING : cannot compute similarity with no input []
2017-04-27 00:55:28,163 : WARNING : cannot compute similarity with no input [u'FOREIGN', u'FOREIGN', u'FOREIGN']
2017-04-27 00:55:28,176 : WARNING : cannot compute similarity with no input []
2017-04-27 00:55:28,808 : WARNING : cannot compute similarity with no input []
2017-04-27 00:55:28,884 : WARNING : cannot compute similarity with no input []
2017-04-27 00:55:29,051 : WARNING : cannot compute similarity with no input [u'ERRO']
2017-04-27 00:55:29,062 : WARNING : cannot compute similarity with no input [u'N

CPU times: user 3.44 s, sys: 44.2 ms, total: 3.48 s
Wall time: 3.46 s


In [128]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [130]:
%%time
model_trainer.fit(train_centroids, train_w2v_data['extraversion'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib64/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/usr/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.................................................................

In [15]:
joblib.dump(model, '/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model3.pkl')

['/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model3.pkl']

In [ ]:
yp = model.predict(test_centroids)
yt = test_w2v_data['extraversion']

In [ ]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

In [ ]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

In [ ]:
accuracy_score(yt, yp)